In [1]:
import pandas as pd

In [2]:
# show all columns
pd.set_option('display.max_columns', None)

## transactions

In [3]:
df_transactions = pd.read_csv('data/raw/bank_small/transactions.csv')

In [4]:
df_transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121457 entries, 0 to 121456
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   tran_id         121457 non-null  int64  
 1   orig_acct       121457 non-null  int64  
 2   bene_acct       121457 non-null  int64  
 3   tx_type         121457 non-null  object 
 4   base_amt        121457 non-null  float64
 5   tran_timestamp  121457 non-null  object 
 6   is_sar          121457 non-null  bool   
 7   alert_id        121457 non-null  int64  
dtypes: bool(1), float64(1), int64(4), object(2)
memory usage: 6.6+ MB


In [5]:
df_transactions.describe()

,tran_id,orig_acct,bene_acct,base_amt,alert_id
count,121457.00000,121457.000000,121457.000000,121457.000000,121457.000000
mean,60729.00000,749.612949,186.781544,540.937783,-0.996995
std,35061.76016,478.960164,306.150076,263.058184,0.147206
min,1.00000,0.000000,0.000000,0.250000,-1.000000
25%,30365.00000,282.000000,25.000000,312.720000,-1.000000
50%,60729.00000,868.000000,52.000000,537.790000,-1.000000
75%,91093.00000,1164.000000,166.000000,768.250000,-1.000000
max,121457.00000,1445.000000,1442.000000,999.990000,9.000000


In [6]:
df_transactions.head()

,tran_id,orig_acct,bene_acct,tx_type,base_amt,tran_timestamp,is_sar,alert_id
0,1,982,23,TRANSFER,501.95,2017-01-01T00:00:00Z,False,-1
1,2,1107,104,TRANSFER,248.90,2017-01-01T00:00:00Z,False,-1
2,3,659,107,TRANSFER,552.61,2017-01-01T00:00:00Z,False,-1
3,4,620,10,TRANSFER,945.10,2017-01-01T00:00:00Z,False,-1
4,5,504,103,TRANSFER,604.33,2017-01-01T00:00:00Z,False,-1


In [22]:
df_transactions['alert_id'].unique()

array([-1,  2,  0,  1,  4,  7,  9,  5,  3,  6,  8])

In [14]:
df_transactions['is_sar'].value_counts().head()

is_sar
False    121392
True         65
Name: count, dtype: int64

In [7]:
df_alert_tran = pd.read_csv('data/raw/bank_small/alert_transactions.csv')
df_alert_tran.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65 entries, 0 to 64
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   alert_id        65 non-null     int64  
 1   alert_type      65 non-null     object 
 2   is_sar          65 non-null     bool   
 3   tran_id         65 non-null     int64  
 4   orig_acct       65 non-null     int64  
 5   bene_acct       65 non-null     int64  
 6   tx_type         65 non-null     object 
 7   base_amt        65 non-null     float64
 8   tran_timestamp  65 non-null     object 
dtypes: bool(1), float64(1), int64(4), object(3)
memory usage: 4.3+ KB


In [8]:
df_alert_tran.describe()

,alert_id,tran_id,orig_acct,bene_acct,base_amt
count,65.000000,65.000000,65.000000,65.000000,65.000000
mean,4.615385,39066.907692,520.969231,428.938462,495.513385
std,3.019170,25388.803434,404.989312,434.831536,249.015206
min,0.000000,23.000000,8.000000,9.000000,108.680000
25%,2.000000,18113.000000,220.000000,49.000000,312.520000
50%,4.000000,33945.000000,349.000000,241.000000,459.010000
75%,7.000000,56106.000000,796.000000,757.000000,659.810000
max,9.000000,103602.000000,1416.000000,1393.000000,997.920000


In [9]:
df_alert_tran.head()

,alert_id,alert_type,is_sar,tran_id,orig_acct,bene_acct,tx_type,base_amt,tran_timestamp
0,2,fan_in,True,23,424,38,TRANSFER,615.19,2017-01-01T00:00:00Z
1,0,fan_in,True,193,1238,9,TRANSFER,335.76,2017-01-01T00:00:00Z
2,1,fan_in,True,222,108,53,TRANSFER,411.40,2017-01-01T00:00:00Z
3,4,fan_out,True,8516,349,1187,TRANSFER,678.05,2017-02-07T00:00:00Z
4,0,fan_in,True,10914,93,9,TRANSFER,556.89,2017-02-17T00:00:00Z


In [16]:
df_transactions['tran_id'].isin(df_alert_tran['tran_id']).mean()

np.float64(0.0005351688251809282)

In [20]:
df_alert_tran['alert_id'].unique()

array([2, 0, 1, 4, 7, 9, 5, 3, 6, 8])

In [18]:
df_alert_tran['tran_id'].nunique()

65

In [10]:
df_accs = pd.read_csv('data/raw/bank_small/accounts.csv')
df_accs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1446 entries, 0 to 1445
Data columns (total 24 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   acct_id           1446 non-null   int64  
 1   dsply_nm          1446 non-null   object 
 2   type              1446 non-null   object 
 3   acct_stat         1446 non-null   object 
 4   acct_rptng_crncy  1446 non-null   object 
 5   prior_sar_count   1446 non-null   bool   
 6   branch_id         1446 non-null   int64  
 7   open_dt           1446 non-null   int64  
 8   close_dt          1446 non-null   int64  
 9   initial_deposit   1446 non-null   float64
 10  tx_behavior_id    0 non-null      float64
 11  bank_id           1446 non-null   object 
 12  first_name        1446 non-null   object 
 13  last_name         1446 non-null   object 
 14  street_addr       1446 non-null   object 
 15  city              1446 non-null   object 
 16  state             1446 non-null   object 


In [11]:
df_accs.describe()

,acct_id,branch_id,open_dt,close_dt,initial_deposit,tx_behavior_id,zip,lon,lat
count,1446.000000,1446.0,1446.0,1446.0,1446.000000,0.0,1446.000000,1446.000000,1446.000000
mean,722.500000,1.0,0.0,1000000.0,75187.784862,NaN,50511.727524,-5.030429,-0.973679
std,417.568557,0.0,0.0,0.0,14435.349809,NaN,28733.281620,103.660898,51.420080
min,0.000000,1.0,0.0,1000000.0,50008.700000,NaN,529.000000,-179.382567,-89.981436
25%,361.250000,1.0,0.0,1000000.0,62779.435000,NaN,25387.250000,-94.046212,-45.088875
50%,722.500000,1.0,0.0,1000000.0,75351.230000,NaN,50988.500000,-4.488273,-1.765097
75%,1083.750000,1.0,0.0,1000000.0,87723.045000,NaN,75301.750000,84.321890,42.884752
max,1445.000000,1.0,0.0,1000000.0,99972.500000,NaN,99936.000000,179.653148,89.976049


In [12]:
df_accs.head()

,acct_id,dsply_nm,type,acct_stat,acct_rptng_crncy,prior_sar_count,branch_id,open_dt,close_dt,initial_deposit,tx_behavior_id,bank_id,first_name,last_name,street_addr,city,state,country,zip,gender,birth_date,ssn,lon,lat
0,0,C_0,I,A,USD,False,1,0,1000000,96646.86,NaN,bank,Steven,Collins,48764 Howard Forge Apt. 421,Vanessaside,VT,US,79393,Male,2009-04-16,076-88-5410,73.479690,60.273842
1,1,C_1,I,A,USD,False,1,0,1000000,80707.36,NaN,bank,Samantha,Cook,9387 Grimes Green Apt. 801,Pagetown,RI,US,65195,Female,2009-05-08,334-91-1032,-77.421639,62.340004
2,2,C_2,I,A,USD,False,1,0,1000000,52101.10,NaN,bank,Robin,Fernandez,711 Golden Overpass,West Andreaville,OH,US,44115,Female,2018-10-05,618-71-9628,-25.565183,29.450832
3,3,C_3,I,A,USD,False,1,0,1000000,90324.40,NaN,bank,Steven,Leblanc,9342 Lori Bypass Suite 711,East Sandra,MP,US,11019,Male,1983-07-15,700-51-8595,-32.020057,50.066962
4,4,C_4,I,A,USD,False,1,0,1000000,91267.86,NaN,bank,Lorraine,Bruce,969 Hayes Shore,Christopherville,NM,US,77450,Female,1943-01-14,017-94-4441,-117.114006,-30.820575
